## **AB Testing Product Case Study**

##Introduction

The aim of this A/B Testing project for Buy Now Pay Later (BNPL) is to evaluate the impact of BNPL offerings on user behavior and business metrics. The key objectives include:

1. Assessing Conversion Rate
- Hypothesis:
Null (H₀): There is no difference in conversion rates between the control and treatment groups.
Alternative (H₁): The treatment group (BNPL adoption) has a higher conversion rate.
- Method: Perform a t-test/z-test to determine statistical significance.
2. Evaluating Average Order Value (AOV) Changes
- Hypothesis:
Null (H₀): There is no difference in AOV between groups.
Alternative (H₁): The treatment group (BNPL users) has a higher AOV.
- Method: Conduct a t-test (for numerical order value data) to compare means.
3. Analyzing User Behavior by Segments
- Categories:
        - Region: Do users in certain locations prefer BNPL more?
        - Device Type: Are mobile users more likely to use BNPL?
        - Payment Method: Does BNPL increase conversion compared to other payment options?

- Method: Perform a chi-square test to analyze the distribution of BNPL adoption across different user segments.

Once we have results, we can come up with business recommendations and insights.

##Data Sourcing

The dataset used for this analysis was taken from ChatGPT using prompting of variables.

In [5]:
#Importing Libraries I will use in this project
import pandas as pd


df = pd.read_csv('ab_testing_bnpl.csv')
df.head()

,user_id,group,converted,order_value,region,device_type,payment_method
0,1,Control,0,45.588385,Asia,Mobile,Debit Card
1,2,Treatment,0,93.447195,Europe,Desktop,BNPL
2,3,Treatment,0,97.578297,North America,Desktop,Credit Card
3,4,Treatment,0,64.274116,North America,Mobile,BNPL
4,5,Control,0,81.524587,Europe,Mobile,Credit Card


In [6]:
#checking the data shape
df.shape

(500000, 7)

There are 500000 users for our experiment.

In [7]:
df.duplicated().sum()

0

No duplicate users

In [8]:
df.isna().sum()

,0
user_id,0
group,0
converted,0
order_value,0
region,0
device_type,0
payment_method,0


No Missing Records

In [11]:
df.groupby(by='group')['user_id'].count()

,user_id
group,
Control,249555
Treatment,250445


Close to 50% of users in treatment and control groups.

Moving into Evaluation Metrics using AB Testing.
---
Lets consider Continuous Variable metrics : Conversion Rate and Average Over Value with BNPL Adoption.



In [12]:
df.groupby(by='group')['converted'].mean()

,converted
group,
Control,0.119012
Treatment,0.139544


Data Distribution based on BNPL Option for Control & Treatment Groups:
1.   Control Group : Conversion is 12%
2.   Treatment Group : Conversion is 14%

In [13]:
df.groupby(by='group')['order_value'].mean()

,order_value
group,
Control,79.922412
Treatment,84.983135


Data Distribution based on Average Order Value for Control & Treatment Groups :
1.   Control Group : Average Order Value (AOV) is 80$
2.   Treatment Group : Average Order Value (AOV) is 85%

#Conversion Rate
---
As we see 'converted' variable as a binary value variable, a two-proportion z-test is useful to determine if the conversion rates between both groups are  significantly different.

In [ ]:
#from statsmodels.stats.proportion import proportions_ztest

In [ ]:
#pip install statsmodels if you dont have statsmodel library install it as its not built in python library

In [20]:
from statsmodels.stats.proportion import proportions_ztest

conversions = df.groupby('group')['converted'].sum()
print(conversions)
n_users = df.groupby('group')['user_id'].count()
print(n_users)

z_stats,p_value = proportions_ztest(conversions,n_users)
print(f'Z-Statistics:{z_stats}, P-Value:{p_value}')

group
Control      29700
Treatment    34948
Name: converted, dtype: int64
group
Control      249555
Treatment    250445
Name: user_id, dtype: int64
Z-Statistics:-21.634801849859638, P-Value:8.451165815521004e-104


###Interpretation:
- As we see large negative Z-score which tells that the conversion rate in the Treatment group (BNPL offered) is significantly higher than in the Control group.
- The P-value is much smaller than 0.05, meaning we reject the null hypothesis (H₀).
---
With both of this strongly we can say that offering BNPL significantly impacts conversion rates.

#Average Order Value
---
As we see 'order_value' variable is a continuous variable,to compare the means of two groups most approciate test to use would be t-test.

In [26]:
from scipy.stats import ttest_ind

control_group_order_value = df[df['group'] == 'Control']['order_value']
treatment_group_order_value = df[df['group'] == 'Treatment']['order_value']

t_stats,p_value = ttest_ind(control_group_order_value,treatment_group_order_value)
print(f'T-Statistics:{t_stats}, P-Value:{p_value}')

T-Statistics:-85.0421825217005, P-Value:0.0


###Interpretation:
- We see large -ve value in t stats which tells that the AOV in the Treatment group (BNPL users) is significantly higher than in the Control group.
- The P-value is much smaller than 0.05, meaning we reject the null hypothesis (H₀).
---
This strongly suggests that offering Buy Now, Pay Later (BNPL) has a significant impact on Average Order Value , indicating chances of increase in revenue. Well,need to check other constraints aswell though.

#User Behaviour By Region
    --- Region: Do users in certain locations prefer BNPL more?
---
We see that 'converted' is binary categorical variable and region is categorical varible, so Chi-Square Test would be best way to check statistical signficance.

In [27]:
df.groupby('region')['converted'].mean()

,converted
region,
Asia,0.127918
Europe,0.129607
North America,0.130096


In [31]:
import scipy.stats as stats
contingency_table = pd.crosstab(df['region'], df['converted'])
print(contingency_table)
chi2_stat, p_value, dof, expected = stats.chi2_contingency(contingency_table)
print(f"Chi-Square Statistic: {chi2_stat},P-Value: {p_value}")

converted           0      1
region                      
Asia           130807  19187
Europe         130525  19436
North America  174020  26025
Chi-Square Statistic: 3.793656906392925,P-Value: 0.15004373612833674


###Interpretation:
- Since p-value (0.150) is greater than 0.05, we fail to reject the null hypothesis (H₀).
- This means there is no statistically significant difference in conversion rates across regions,the conversion rate does not vary between North America, Europe, and Asia in both groups.

#User Behaviour By Device Type
    --- Device Type: Are mobile users more likely to use BNPL?
---
We see that 'converted' is binary categorical variable and Device Type is categorical varible, so Chi-Square Test would be best way to check statistical signficance.    
      

In [32]:
df.groupby('device_type')['converted'].mean()

,converted
device_type,
Desktop,0.129465
Mobile,0.129004
Tablet,0.130534


In [33]:
contingency_table = pd.crosstab(df['device_type'], df['converted'])
print(contingency_table)
chi2_stat, p_value, dof, expected = stats.chi2_contingency(contingency_table)
print(f"Chi-Square Statistic: {chi2_stat},P-Value: {p_value}")

converted         0      1
device_type               
Desktop      130743  19444
Mobile       261027  38661
Tablet        43582   6543
Chi-Square Statistic: 0.9469761344661907,P-Value: 0.6228260160136767


###Interpretation:
- Since p-value (0.622) is greater than 0.05, we fail to reject the null hypothesis (H₀).
- The conversion rate does not vary between different device types.

#User Behaviour By Device Type
     - Payment Method: Does BNPL increase conversion compared to other payment options?
---
We see that 'converted' is binary categorical variable and Payment Method is categorical varible, so Chi-Square Test would be best way to check statistical signficance.   

In [34]:
df.groupby('payment_method')['converted'].mean()

,converted
payment_method,
BNPL,0.130520
Credit Card,0.128605
Debit Card,0.129633


In [35]:
contingency_table = pd.crosstab(df['payment_method'], df['converted'])
print(contingency_table)
chi2_stat,p_value,dof,expected = stats.chi2_contingency(contingency_table)
print(f"Chi-Square Statistic: {chi2_stat},P-Value: {p_value}")

converted            0      1
payment_method               
BNPL             87108  13076
Credit Card     218118  32191
Debit Card      130126  19381
Chi-Square Statistic: 2.544971442541471,P-Value: 0.2801344195268534


###Interpretation:
- Since p-value (0.28) is greater than 0.05, we fail to reject the null hypothesis (H₀).
- The conversion rate does not vary between different payment methods.

#Final Takeaways & Business Implications:
- ✅ BNPL is effective in driving higher conversions and increasing AOV.
- ❌ User behavior segmentation did not show significant differences, meaning BNPL adoption is uniform across user groups.